In this Notebook, we aim to learn the contextual embeddings for the Pronoun, and its possible resolutions from the sentence  and frame the Pronoun resolution task as a Supervised Classification task.


---
We obtain the contextual embeddings using BERT and feed this embeddings to a classifier to obtain results.



---
The performance of the model has been tested with GAP dataset.



In [ ]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!git clone https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/transformers

In [16]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForPreTraining
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:

def run_bert(data):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

  model = TFBertForPreTraining.from_pretrained('bert-base-uncased',output_hidden_states = True)

  index = data.index
  columns = ["emb_A", "emb_B", "emb_P", "label"]
  emb = pd.DataFrame(index = index, columns = columns)
  emb.index.name = "ID"
  # print('Length of data__________________',len(data))
  for rowidx in range(0,len(data)):
    
    text = data.loc[rowidx,'Text']
    
    input_ids = tf.constant(tokenizer.encode(text, add_special_tokens=True))[None, :]  # Batch size 1
    
    # Tokenize our sentence with the BERT tokenizer.
    tokenized_text = tokenizer.tokenize(text)
  
    # Print out the tokens.
    # print(tokenized_text)

    outputs = model(input_ids)
    hidden_states = outputs[2] # Taking the hidden layers

    A = data.loc[rowidx,'A'].lower()
    # print('A---------',A)
    B = data.loc[rowidx,'B'].lower()
    # print('B---------',B)
    P = data.loc[rowidx,'Pronoun'].lower()
    # print('P---------',P,'\n')

    A_embedding = np.zeros(768)
    B_embedding = np.zeros(768)
    P_embedding = np.zeros(768)

    #print([A_embedding,B_embedding,P_embedding])

    A_count = 0   # Used to take average if there are more than one occurence of A in a single sentence
    B_count = 0   # Used to take average if there are more than one occurence of B in a single sentence
    P_count = 0   # Used to take average if there are more than one occurence of P in a single sentence


    for layers in range(1,len(hidden_states)):
      for sentences in hidden_states[layers]:
        idx = 0
        for tokens in range(1,len(sentences)-1):
          tok = tokenized_text[idx].strip('#')
          idx = idx + 1
          tok = tok.lower()
          #if(rowidx == 4):
            #print(tok)
          if tok in A:
            #print(sentences[tokens])
            if(A.index(tok) > 0): # check for firstname and last name
              continue

            A_embedding += sentences[tokens]  # calculating the embedding for A
            A_count += 1

          if tok in B:
            #print(sentences[tokens])
            if(B.index(tok) > 0):
              continue

            B_embedding += sentences[tokens]  # calculating the embedding for B
            B_count += 1

          if tok in P:
            if(P.index(tok) > 0):
              continue
            P_embedding += sentences[tokens]  # calculating the embedding for P
            P_count += 1

    #if(rowidx == 4):
   #   print('Inside A_____________',A_count)
    # print('Inside A_____________',A_embedding)
    A_embedding /= A_count
    
    #if(rowidx == 4):
     # print('Inside B_____________',B_count)
    # print('Inside B_____________',B_embedding)
    B_embedding /= B_count

   # if(rowidx == 4):
    #  print('Inside P_____________',P_count)
    # print('Inside P_____________',P_embedding)
    P_embedding /= P_count
      
    label = "Neither"
    if (data.loc[rowidx,"A-Coref"] == True):
      label = "A"
    if (data.loc[rowidx,"B-Coref"] == True):
      label = "B"
    A_embedding = np.array(A_embedding)
    B_embedding = np.array(B_embedding)
    P_embedding = np.array(P_embedding)
    # print(A_embedding.shape, B_embedding.shape,P_embedding.shape)

    emb.iloc[rowidx] = [A_embedding,B_embedding,P_embedding,label]

  return emb

In [4]:
data = pd.read_csv('/content/Cleaned_Dataset.csv')
data = data.drop('Unnamed: 10',axis=1)
data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-Coref,B,B-offset,B-Coref
0,0,"After the war, Herbert attended the University...",he,8,Herbert,16,True,Beverly,76,False
1,1,Herbert was appalled to learn of senator McCar...,he,140,Herbert,1,True,McCarthy,42,False
2,2,Herbert has attracted a sometimes fanatical fa...,he,98,Herbert,1,True,Herbert,1,False
3,3,"In 1887, after the death of his brother and a ...",he,17,Norris,75,False,Julian,99,True
4,4,"The publication of Buechner's third novel, The...",he,2,Buechner,20,True,Buechner,20,False


In [5]:
print(len(data))

6570


In [6]:
data = data.sample(frac=1).reset_index(drop=True)
train_data = data[:4930]
print(len(train_data))
test_data = data[4930:]
print(len(test_data))

4930
1640


In [ ]:
train_emb = run_bert(train_data)
train_emb.to_json("Bert_embs_train.json", orient = 'columns')

In [37]:
test_data = test_data.reset_index(drop=True)
test_data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-Coref,B,B-offset,B-Coref
0,3300,"Mankiewicz's film of Anthony's Sleuth, which T...",he,47,Mankiewicz,1,True,Anthony,22,False
1,4167,Despite his misgivings about its defensibility...,he,93,Johnston,49,True,Beauregard,68,False
2,1659,"In September 2008, on the eve of the Labour Pa...",she,86,Rowling,63,True,Gordon,177,False
3,4391,"Wagner had to flee, first visiting Paris and t...",he,47,Wagner,1,True,Alexander ller,115,False
4,4392,Wagner scholars have argued that Schopenhauer'...,he,147,Wagner,1,True,Schopenhauer,34,False


In [ ]:
test_emb = run_bert(test_data)
test_emb.to_json("bert_ebms_test.json", orient = 'columns')

In [ ]:
!pip install xgboost


## XGBoost Classifier

In [32]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier

In [81]:
def parse_json(embeddings):
  embeddings.sort_index(inplace = True)
  X = np.zeros((len(embeddings),3*768))
  Y = np.zeros((len(embeddings), 3))
  label = embeddings["label"]
  # Concatenate features
  for i in range(len(embeddings)):
    A = np.array(embeddings.loc[i,"emb_A"])
    B = np.array(embeddings.loc[i,"emb_B"])
    P = np.array(embeddings.loc[i,"emb_P"])
    X[i] = np.concatenate((A,B,P))
    if np.sum(np.isnan(X[i])):
      del label[i]

  
  remove_test  = [row for row in range(len(X)) if np.sum(np.isnan(X[row]))]
  X = np.delete(X, remove_test, 0)
  return X, label

In [82]:
test = pd.read_json("/content/Bert_embs_test.json")
X_test, Y_test = parse_json(test)

In [83]:
training_data = pd.read_json("/content/Bert_embs_train.json")
X_train, Y_train = parse_json(training_data)

In [84]:
print(len(X_train),len(Y_train))

4688 4688


In [87]:
model = XGBClassifier()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.78%


In [88]:
from sklearn.metrics import classification_report
target_names = ['A', 'B']
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           A       0.81      0.89      0.84      1062
           B       0.69      0.55      0.61       500

    accuracy                           0.78      1562
   macro avg       0.75      0.72      0.73      1562
weighted avg       0.77      0.78      0.77      1562



In [45]:
import pickle
filename = 'CompleteXGB.sav'
pickle.dump(model, open(filename, 'wb'))

# Testing on GAP data

In [17]:
!wget https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv

--2020-11-10 22:58:24--  https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1075889 (1.0M) [text/plain]
Saving to: ‘gap-test.tsv’

gap-test.tsv        100%[===================>]   1.03M  --.-KB/s    in 0.05s   

2020-11-10 22:58:24 (20.5 MB/s) - ‘gap-test.tsv’ saved [1075889/1075889]



In [26]:
gap_data = pd.read_csv('/content/gap-test.tsv',sep = '\t')
gap_data = gap_data.rename(columns={"A-coref": "A-Coref", "B-coref": "B-Coref"})
gap_data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-Coref,B,B-offset,B-Coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [27]:
gap_test = run_bert(gap_data)
gap_test.to_json("gap_test.json", orient = 'columns')

All model checkpoint layers were used when initializing TFBertForPreTraining.

All the layers of TFBertForPreTraining were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in true_divide


In [89]:
gap_test_data = pd.read_json("/content/gap_test.json")
GX_test, GY_test = parse_json(gap_test_data)

In [90]:
gy_pred = model.predict(GX_test)
accuracy = accuracy_score(GY_test, gy_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 54.71%


In [91]:
from sklearn.metrics import classification_report
target_names = ['A', 'B','Neither']
print(classification_report(GY_test, gy_pred, target_names=target_names))

              precision    recall  f1-score   support

           A       0.57      0.59      0.58       879
           B       0.53      0.65      0.58       831
     Neither       0.00      0.00      0.00       213

    accuracy                           0.55      1923
   macro avg       0.37      0.41      0.39      1923
weighted avg       0.49      0.55      0.51      1923



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
